In [2]:
import os
import librosa
import librosa.display
import pandas as pd

import tensorflow as tf
import keras
import matplotlib as plt
import sklearn


In [3]:
import numpy as np
import pickle

In [4]:
audio_fpath = "../desafio/data/Test/"
audio_clips = os.listdir(audio_fpath)
print("No. of .wav files in audio folder = ",len(audio_clips))

No. of .wav files in audio folder =  600


In [6]:
X_test = []
for clip in audio_clips:
    x , sr = librosa.load(f"../desafio/data/Test/{clip}")
    X = librosa.stft(x)
    Xdb = librosa.amplitude_to_db(abs(X))
    X_test.append(Xdb)

In [14]:
maxshape = 0
for s in X_test:
    if s.shape[1] >= maxshape:
        maxshape = s.shape[1]
X_test[0].shape[0],maxshape

(1025, 515)

In [21]:
datareshape = []
for item in X_test:
    i,j = item.shape
    fill_left = np.floor((maxshape-j)/2)
    fill_right = np.ceil((maxshape-j)/2)
    datareshape.append(np.pad(item, ((0,0),(int(fill_left)+11,int(fill_right)+11)), mode='constant', constant_values=np.min(item)))

In [22]:
nptest = np.asarray(datareshape)
nptest.shape

(600, 1025, 537)

In [24]:
gender_model = keras.models.load_model("../Redes/desafio/models/Gender1")

country_model = keras.models.load_model("../Redes/desafio/models/Country1")

In [26]:
gen = gender_model.predict(nptest)

19/19 [==============================] - 3s 136ms/step


In [30]:
gen = gen.round()

In [31]:
count = country_model.predict(nptest)

19/19 [==============================] - 1s 22ms/step


In [60]:
df = []
for i in range(len(gen)):
    idx = np.argmax(count[i])+2
    df.append([f"{str(int(gen[i]))} {idx}"])

In [65]:
df = pd.DataFrame(df,columns=["Expected"])

In [64]:
names = pd.read_csv('../desafio/data/Test.csv')

In [69]:
final = pd.concat([names,df],axis=1)

In [74]:
final.to_csv("Test_submit.csv",index=False)

Index(['Id', 'Expected'], dtype='object')